In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import pillow_heif
from PIL.ExifTags import TAGS
import piexif
from pillow_heif import register_heif_opener



os.chdir("/home/michael/AmbientLight/")


In [55]:
image_paths = ["test_photos_heic/table_-2.HEIC", "test_photos_heic/table_0.HEIC", "test_photos_heic/table_+2.HEIC"]
image = Image.open('/home/michael/AmbientLight/test_photos_heic/table-2.HEIC')
register_heif_opener()



In [3]:
def decode_exif(exif_bytes):
    exif_data = {}
    if not exif_bytes:
        return exif_data  # No EXIF data found, return empty dictionary

    try:
        exif_dict = piexif.load(exif_bytes)

        # Flatten the EXIF data from all IFDs
        for ifd in exif_dict:
            for tag, value in exif_dict[ifd].items():
                decoded_tag = TAGS.get(tag, tag)
                exif_data[decoded_tag] = value

    except Exception:
        pass  # Suppress errors silently
    return exif_data

def extract_specific_metadata(image_paths):
    # Define the EXIF tags to extract
    desired_tags = [
        "ExposureTime",
        "FNumber",
        "ISOSpeedRatings",
        "BrightnessValue",
        "ExposureBiasValue"
    ]

    metadata_list = []

    for image_path in image_paths:
        metadata = {"filename": image_path}
        try:
            # Read HEIC file and metadata
            heif_file = pillow_heif.read_heif(image_path)
            exif_bytes = heif_file.info.get("exif", b"")

            # Decode EXIF bytes
            decoded_exif = decode_exif(exif_bytes)

            # Extract only the desired tags
            for tag in desired_tags:
                value = decoded_exif.get(tag, None)
                metadata[tag] = value

        except Exception:
            # If there's an error, set all desired tags to None
            for tag in desired_tags:
                metadata[tag] = None

        metadata_list.append(metadata)

    return metadata_list

def format_exif_value(tag, value):

    if tag == "ExposureTime" and isinstance(value, tuple):
        return f"{value[0]}/{value[1]}"
    if tag == "FNumber" and isinstance(value, tuple):
        return f"{value[0]}/{value[1]} = {value[0]/value[1]:.1f}"
    if tag in ["BrightnessValue", "ExposureBiasValue"] and isinstance(value, tuple):
        return f"{value[0]}/{value[1]} = {value[0]/value[1]:.2f}"
    return str(value)


image_paths = [
    "test_photos_heic/table-2.HEIC",
    "test_photos_heic/table0.HEIC",
    "test_photos_heic/table+2.HEIC"
]

metadata_list = extract_specific_metadata(image_paths)

for metadata in metadata_list:
    print(f"Metadata for {metadata['filename']}:")
    for key in ["ExposureTime", "FNumber", "ISOSpeedRatings", "BrightnessValue", "ExposureBiasValue"]:
        formatted_value = format_exif_value(key, metadata.get(key))
        print(f"  {key}: {formatted_value}")
    print("\n")


Metadata for test_photos_heic/table-2.HEIC:
  ExposureTime: 1/120
  FNumber: 11/5 = 2.2
  ISOSpeedRatings: 125
  BrightnessValue: 16317/6947 = 2.35
  ExposureBiasValue: -2/1 = -2.00


Metadata for test_photos_heic/table0.HEIC:
  ExposureTime: 1/60
  FNumber: 1244236/699009 = 1.8
  ISOSpeedRatings: 250
  BrightnessValue: 30755/15562 = 1.98
  ExposureBiasValue: 0/1 = 0.00


Metadata for test_photos_heic/table+2.HEIC:
  ExposureTime: 1/60
  FNumber: 1244236/699009 = 1.8
  ISOSpeedRatings: 800
  BrightnessValue: 23423/12120 = 1.93
  ExposureBiasValue: 2/1 = 2.00




In [4]:
metadata_list

[{'filename': 'test_photos_heic/table-2.HEIC',
  'ExposureTime': (1, 120),
  'FNumber': (11, 5),
  'ISOSpeedRatings': 125,
  'BrightnessValue': (16317, 6947),
  'ExposureBiasValue': (-2, 1)},
 {'filename': 'test_photos_heic/table0.HEIC',
  'ExposureTime': (1, 60),
  'FNumber': (1244236, 699009),
  'ISOSpeedRatings': 250,
  'BrightnessValue': (30755, 15562),
  'ExposureBiasValue': (0, 1)},
 {'filename': 'test_photos_heic/table+2.HEIC',
  'ExposureTime': (1, 60),
  'FNumber': (1244236, 699009),
  'ISOSpeedRatings': 800,
  'BrightnessValue': (23423, 12120),
  'ExposureBiasValue': (2, 1)}]

In [ ]:
  ExposureTime: (1, 120) # 1/120
  FNumber: (11, 5) # 11/5 = 2.2
  
  
  ISOSpeedRatings: 125 # straight up

  BrightnessValue: (16317, 6947)
  ExposureBiasValue: (-2, 1)
  
      




In [ ]:
# List of exposure images and their exposure times
image_paths = ["test_photos_heic/table_-2.HEIC", "test_photos_heic/table_0.HEIC", "test_photos_heic/table_+2.HEIC"]

exposure_times = np.array([1/30.0, 1/60.0, 1/125.0], dtype=np.float32)  # Corresponding exposure times

# Load images
images = [cv2.imread(img) for img in image_paths]
